In [32]:
import json

import pooch
import pandas as pd
import xarray as xr

In [33]:
key = "qPwOeIrU-2501-PXXKVD-1011"
sdate = "19680101"
edate = "20241231"
url=f"https://www.nifs.go.kr/OpenAPI_json?id=sooList&key={key}&sdate={sdate}&edate={edate}"


In [34]:
fname_soo = pooch.retrieve(
    url=url,
    known_hash=None,
)


In [35]:
with open(fname_soo, "r", encoding="euc-kr") as file:
    raw_soo = json.load(file)

In [36]:
data_soo = pd.DataFrame(raw_soo["body"]["item"])

In [37]:
data_soo["obs_dtm"] = pd.to_datetime(data_soo["obs_dtm"])

In [38]:
data_soo.columns

Index(['nut_no3_n', 'sln_cde', 'lon', 'wtr_trn', 'wtr_tmp', 'nut_no2_n',
       'gru_nam', 'res_vsl_nm', 'nut_sio2_si', 'wtr_dep', 'dox', 'sta_cde',
       'qc_wtr', 'qc_sal', 'obs_dtm', 'sal', 'atm', 'nut_ph', 'qc_dox',
       'nut_po4_p', 'lat'],
      dtype='object')

In [39]:
# xarray Dataset 생성 함수
def dataframe_to_xarray(col):
    return xr.DataArray(
        data_soo[col],
        dims=["time"],
        coords={"time": data_soo["obs_dtm"],
                "x": ("time", data_soo["lon"]),
                "y": ("time", data_soo["lat"]),
                "z": ("time", data_soo["wtr_dep"])}
    )


In [40]:
# xarray Dataset 생성
data = xr.Dataset({col: dataframe_to_xarray(col) for col in data_soo.columns if col not in ["obs_dtm"]})

print(data)

<xarray.Dataset> Size: 2MB
Dimensions:      (time: 10342)
Coordinates:
  * time         (time) datetime64[ns] 83kB 1968-12-18T14:25:00 ... 1968-01-1...
    x            (time) object 83kB '125.4167' '125.4167' ... '128.8283'
    y            (time) object 83kB '36.925' '36.925' ... '34.3733' '34.3733'
    z            (time) object 83kB '0' '10' '20' '30' ... '30' '50' '75' '90'
Data variables: (12/20)
    nut_no3_n    (time) object 83kB None None None None ... None None None None
    sln_cde      (time) object 83kB '307' '307' '307' ... '206' '206' '206'
    lon          (time) object 83kB '125.4167' '125.4167' ... '128.8283'
    wtr_trn      (time) object 83kB '4.5' '4.5' '4.5' '4.5' ... '13' '13' '13'
    wtr_tmp      (time) object 83kB '10.3' '10.84' '10.58' ... '10.98' '10.52'
    nut_no2_n    (time) object 83kB None None None None ... None None None None
    ...           ...
    sal          (time) object 83kB '32.85' '32.79' '32.79' ... None None None
    atm          (time) ob